In [3]:
import os
root_proj = '/Users/luo/OneDrive/GitHub/Glacier-in-RGI1305/data/icesat/glah14-download/2009/tile-27-91'
os.chdir(root_proj)


In [4]:
import h5py 
import glob
import numpy as np


In [40]:
bounding_box = '91,27,92,28' 


'91'